In [1]:
!nvidia-smi

Mon Oct 16 13:13:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.00       Driver Version: 517.00       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   55C    P5    25W /  N/A |    939MiB /  6144MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x1f5fff1f490>)

In [4]:
torch.cuda.device_count()

1

In [6]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())
    ]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [8]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [9]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [11]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y, Y.device

(tensor([[0.9206, 0.9959, 0.3249],
         [0.6247, 0.6413, 0.6559]]),
 device(type='cpu'))

In [16]:
Z = Y.cuda(0)
print(Y, Y.device)
print(Z)

tensor([[0.9206, 0.9959, 0.3249],
        [0.6247, 0.6413, 0.6559]]) cpu
tensor([[0.9206, 0.9959, 0.3249],
        [0.6247, 0.6413, 0.6559]], device='cuda:0')


In [19]:
X + Z

tensor([[1.9206, 1.9959, 1.3249],
        [1.6247, 1.6413, 1.6559]], device='cuda:0')

In [22]:
try:
    print(X + Y)
except Exception as e:
    print(e)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


In [23]:
Z.cuda(0) is Z

True

In [24]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.0506],
        [-0.0506]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [25]:
net[0].weight.data.device

device(type='cuda', index=0)